# Model Correction

```{margin}
```{important}
This tutorial assumes that `BuildingMOTIF` has already been installed in the local environment.
```

The purpose of this tutorial is to learn how to fix a model that fails validation using BuildingMOTIF Templates to automate the correction process.

## Setup

Like the previous tutorial, we'll create an in-memory BuildingMOTIF instance, load the model, and load some libraries. We'll also load the manifest from the previous tutorial.

```{margin}
```{warning}
Currently, libraries in `../../buildingmotif/libraries/` are *included* and libraries in `../../libraries/` are *excluded* from the [BuildingMOTIF Python package](https://pypi.org/project/buildingmotif/) (available by cloning, downloading, or forking the repository). See https://github.com/NREL/BuildingMOTIF/issues/133. 
```

In [1]:
from rdflib import Namespace
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
from buildingmotif.namespaces import BRICK # import this to make writing URIs easier

# in-memory instance
bm = BuildingMOTIF("sqlite://")

# create the namespace for the building
BLDG = Namespace('urn:bldg/')

# create the building model
model = Model.create(BLDG, description="This is a test model for a simple building")

# load tutorial 2 model and manifest
model.graph.parse("tutorial2_model.ttl", format="ttl")
manifest = Library.load(ontology_graph="tutorial2_manifest.ttl")

# load libraries included with the python package
constraints = Library.load(ontology_graph="../../buildingmotif/libraries/constraints/constraints.ttl")

# load libraries excluded from the python package (available from the repository)
brick = Library.load(ontology_graph="../../libraries/brick/Brick-subset.ttl")
g36 = Library.load(directory="../../libraries/ashrae/guideline36")

## Model Validation

Let's validate the model again to see what's causing the failure.

In [2]:
# gather shape collections into a list for ease of use
shape_collections = [
    brick.get_shape_collection(),
    constraints.get_shape_collection(),
    manifest.get_shape_collection(),
    g36.get_shape_collection(),
]

# pass a list of shape collections to .validate()
validation_result = model.validate(shape_collections)
print(f"Model is valid? {validation_result.valid}")

# print reasons
for diff in validation_result.diffset:
    print(f" - {diff.reason()}")

Model is valid? False


 - urn:bldg/Core_ZN-PSC_AC needs between 1 and 1 instances of urn:ashrae/g36/4.8/sz-vav-ahu/sa-fan on path https://brickschema.org/schema/Brick#hasPart


## Model Correction with Templates

The model is failing because the AHU doesn't have the minimum number of supply fans associated with it. We *could* add the fan explicitly by adding those triples to the model like we've done previously, but we can also ask BuildingMOTIF to generate new templates that explicitly prompt us for the missing information. We can also take a closer look at the first autogenerated template

In [3]:
# create a new library to hold these generated templates
generated_templates = Library.create("my-autogenerated-templates")
for diff in validation_result.diffset:
    diff.resolve(generated_templates)

# print some of the autogenerated template
for templ in generated_templates.get_templates():
    templ = templ.inline_dependencies()
    print(f"Name (autogenerated): {templ.name}")
    print(f"Parameters (autogenerated): {templ.parameters}")
    print("Template body (autogenerated):")
    print(templ.body.serialize())
    print('-' * 79)
    break # just to demonstrate the first piece of output

Name (autogenerated): resolvedea1adc3
Parameters (autogenerated): {'p1', 'name', 'p2', 'p3'}
Template body (autogenerated):
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix default6: <urn:ashrae/g36/4.8/sz-vav-ahu/> .

<urn:bldg/Core_ZN-PSC_AC> brick:hasPart <urn:___param___#name> .

<urn:___param___#name> a brick:Supply_Fan,
        default6:sa-fan ;
    brick:hasPoint <urn:___param___#p1>,
        <urn:___param___#p2>,
        <urn:___param___#p3> .

<urn:___param___#p1> a brick:Frequency_Command .

<urn:___param___#p2> a brick:Fan_Status .

<urn:___param___#p3> a brick:Start_Stop_Command .


-------------------------------------------------------------------------------


In this case, the generated templates are fairly simple. They require an input for the name of the supply fan and the names of several missing points. We can loop through each of these generated templates and create the names. Here, we are creating arbitrary names for the points but in a real setting you would likely pull the equipment or point names from an external source like a Building Information Model or BACnet network[^1] (see future tutorials for how to do this!) Another challenge is the fact that we already have a supply fan in the model. Here, we can take advantage of the fact that the name of the fan in the existing model are just the name of the AHU wtih the `-Fan` postfix. The name of the AHU is in the generated templates (see above) so we can just pull out the name of the AHU, add the postfix, and use that as the value for the `name` parameter.

[^1]: https://bacnet.org/

In [4]:
from buildingmotif.namespaces import BRICK
for templ in generated_templates.get_templates():
    templ = templ.inline_dependencies()
    # get name of AHU from template body
    ahu_name = next(templ.body.subjects(predicate=BRICK.hasPart))
    # generate the name of the supply fan
    supply_fan_name = ahu_name + '-Fan'

    # we know from the exploration above that each template has
    # 1 parameter called 'name', which we assign to the name of the supply fan
    bindings = {
        "name": supply_fan_name,
    }
    # the rest of the parameters have random names, so let's generate
    # random URIs for those by putting the name of the parameter at the end
    # of the fan's name
    for p in templ.parameters - {'name'}:
        print(p)
        bindings[p] = supply_fan_name + p
    supply_fan = templ.evaluate(bindings)
    model.add_graph(supply_fan)
    print(f"Added supply fan {supply_fan_name}")

p1
p2
p3
Added supply fan urn:bldg/Core_ZN-PSC_AC-Fan


We use the same code as before to ask BuildingMOTIF if the model is now valid:

In [5]:
validation_result = model.validate(shape_collections)
print(f"Model is valid? {validation_result.valid}")

Model is valid? True


Success! The model is valid with respect to the targeted use case, i.e. the model can support the high-performance sequences of operation for single zone VAV AHUs from ASHRAE Guideline 36. Let's take a look at the validated model and save it for use in future tutorials.

In [6]:
# print model
print(model.graph.serialize())

#save model
model.graph.serialize(destination="tutorial3_model.ttl")

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix default6: <urn:ashrae/g36/4.8/sz-vav-ahu/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<urn:bldg/> a owl:Ontology .

<urn:bldg/Core_ZN-PSC_AC> a brick:AHU ;
    brick:hasPart <urn:bldg/Core_ZN-PSC_AC-Clg_Coil>,
        <urn:bldg/Core_ZN-PSC_AC-Damper>,
        <urn:bldg/Core_ZN-PSC_AC-Fan>,
        <urn:bldg/Core_ZN-PSC_AC-Htg_Coil> .

<urn:bldg/Core_ZN-PSC_AC-Clg_Coil> a brick:Cooling_Coil .

<urn:bldg/Core_ZN-PSC_AC-Damper> a brick:Damper .

<urn:bldg/Core_ZN-PSC_AC-Fan> a brick:Supply_Fan,
        default6:sa-fan ;
    brick:hasPoint <urn:bldg/Core_ZN-PSC_AC-Fanp1>,
        <urn:bldg/Core_ZN-PSC_AC-Fanp2>,
        <urn:bldg/Core_ZN-PSC_AC-Fanp3> .

<urn:bldg/Core_ZN-PSC_AC-Fanp1> a brick:Frequency_Command .

<urn:bldg/Core_ZN-PSC_AC-Fanp2> a brick:Fan_Status .

<urn:bldg/Core_ZN-PSC_AC-Fanp3> a brick:Start_Stop_Command .

<urn:bldg/Core_ZN-PSC_AC-Htg_Coil> a brick:Heating_Coil .




<Graph identifier=6de27308-2590-4f64-a171-f62cb4b304c3 (<class 'rdflib.graph.Graph'>)>